In [8]:
from bs4 import BeautifulSoup
import concurrent.futures
import pandas as pd
import datetime
import requests
import re

In [3]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36",
    "Accept": "application/json",
    "Connection": "keep-alive"}

In [6]:
class EksiScraping:
    case_list = list()
    def __str__(self):
        return "Eksi Sözlük Scraping"

    def createUrls(self, keyword, start_date, end_date):
        urls = list()
        base_url = "https://eksisozluk.com/"
        keyword = f'{keyword}'
        start_date = f'{start_date}'
        end_date = f'{end_date}'
        unix = str(datetime.datetime.utcnow().timestamp())
        url = f'https://eksisozluk.com/basliklar/ara?SearchForm.Keywords={keyword}&SearchForm.Author=&SearchForm.When.From={start_date}&SearchForm.When.To={end_date}&SearchForm.NiceOnly=false&SearchForm.SortOrder=Date&_={unix}'
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        #Girilen keyworde ait alt başlıkların kontrolü. Alt başlık mevcut değilse except bloğu çalışacaktır
        try:
            subheading = [i.get('href').split('?')[0].replace('/', '') for i in
                          soup.find('ul', attrs={'class': 'topic-list partial'}).find_all('a')]
        except AttributeError:
            print("Bu tarih aralığında paylaşım yok")
        else:
            for subhead in subheading:
                response = requests.get(base_url + subhead, headers=headers)
                soup = BeautifulSoup(response.text, 'html.parser')
                if soup.find("div", {"class": "clearfix sub-title-container"}).find("div", {"class": "pager"}):
                    page_count = int(
                        soup.find("div", {"class": "clearfix sub-title-container"}).find("div", {"class": "pager"})[
                            'data-pagecount'])
                    for i in range(1, page_count + 1):
                        urls.append(base_url + subhead + '?p=' + str(i))
                else:
                    urls.append(base_url + subhead)
            return urls


    def transform(url, case_list=case_list):
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")
        entry_divs = soup.find_all("div",{"class":"content"})
        for entry in entry_divs:
            case = {'text':re.sub(r'\r\n','',entry.text).strip(),
                'author':entry.findNext("footer").find_all("a")[0].text,
                'date':entry.findNext("footer").find_all("a")[1].text}
            case_list.append(case)

Eksi = EksiScraping()
urls = Eksi.createUrls('Python', '2022-05-08', '2022-06-08')
if urls is None:
    print('İçerik bulunamadı, lütfen tekrar deneyin.')
    pass
else:
    with concurrent.futures.ThreadPoolExecutor() as executor:
        executor.map(EksiScraping.transform, urls)
    print(EksiScraping.case_list)

[{'text': "ismini aldigi monty python'a gonderme yapilmasinin tutorial'inda tesvik edildigi programlama dili.", 'author': 'skinny dip', 'date': '25.09.2004 00:27'}, {'text': 'bolme islemlerini default olarak integer olarak yapan, bunu duzeltmek icin kodunuzun basina "from __future__ import division" satirini eklemeniz gereken programlama dili. buradan kendisine anandir scientific dil demek istiyorum.', 'author': 'disq', 'date': '17.10.2004 22:10'}, {'text': 'son nesil interpreted dil orneklerinden, bugun itibariyle uzun suredir bekledigimiz series 60 versiyonu yayinlandi. cekmek icin (bkz: http://www.forum.nokia.com/)', 'author': 'cyrus', 'date': '22.12.2004 16:31 ~ 16:57'}, {'text': "gelmiş geçmiş en tatlı, en anlayışlı ve en 'programmer friendly'* programlama dili.", 'author': 'ean', 'date': '04.01.2005 20:33'}, {'text': 'bu leziz dili adam gibi bir interactive shell içinden kullanmanın keyfini yaşamak için: (bkz: ipython)', 'author': 'evreniz', 'date': '28.06.2005 15:42'}, {'text': 

In [10]:
df = pd.DataFrame(EksiScraping.case_list)